<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#uberduck_ml_dev.exec.select_speakers" data-toc-modified-id="uberduck_ml_dev.exec.select_speakers-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>uberduck_ml_dev.exec.select_speakers</a></span></li></ul></div>

In [ ]:
# default_exp exec.select_speakers

# uberduck_ml_dev.exec.select_speakers

In [ ]:
# export
import argparse
from collections import namedtuple
from dataclasses import dataclass
import json
import os
from pathlib import Path
from shutil import copyfile, copytree
import sys
from typing import List, Optional, Set

from tqdm import tqdm

from uberduck_ml_dev.utils.audio import convert_to_wav
from uberduck_ml_dev.utils.utils import parse_vctk

CACHE_LOCATION = Path.home() / Path(".cache/uberduck/uberduck-ml-dev.db")


@dataclass
class Filelist:
    path: str
    speakers: Optional[List[int]] = None


def _get_speaker_ids(filelist: Filelist) -> Set[int]:
    if filelist.speakers:
        return set(filelist.speakers)
    else:
        raise Exception("pass a list of speakerss")


def select_speakers(filelists: List[Filelist], output_filelist: str):
    speaker_id = 0
    with open(output_filelist, "w") as f_out:
        for filelist in tqdm(filelists):
            speaker_ids = _get_speaker_ids(filelist)
            with open(filelist.path, "r") as f_in:
                for line in f_in.readlines():
                    _, _, speaker_id = line.strip().split("|")
                    if int(speaker_id) in speaker_ids:
                        f_out.write(line)


def parse_args(args):
    parser = argparse.ArgumentParser()
    parser.add_argument("--config", help="path to JSON config")
    return parser.parse_args(args)


try:
    from nbdev.imports import IN_NOTEBOOK
except:
    IN_NOTEBOOK = False

if __name__ == "__main__" and not IN_NOTEBOOK:
    args = parse_args(sys.argv[1:])
    if args.config:
        config = json.load(args.config)
        filelists = config["filelists"]
        output_filelist = config["output"]
    else:
        raise Exception("You must pass a config file!")
    select_speakers(filelists, output_filelist)

In [ ]:
args = parse_args(["--config", "foo.json"])
assert args.config == "foo.json"

In [ ]:
# sam stuff
import sqlite3
import os
import pandas as pd


def get_filelist(database, speakerjson):
    """
    Take a list of speakers and create a filelist
    """


def subset_speakers(database, seed, speakerlist=None, nspeakers=None):
    """
    Takes a filelist and saves another filelist with either a random subset of speakers or speakers from the list
    """

In [ ]:
vctk_filelist2, namelist = parse_vctk2(vctk_folder)
print(namelist)

In [ ]:
conn = sqlite3.connect("test.db")

conn.execute(
    """CREATE TABLE DATAINFO
         (ID INT PRIMARY KEY     NOT NULL,
         NAME           TEXT     NOT NULL,
         SOURCE         TEXT     NOT NULL,
         FILELIST       TEXT,
         SPEAKERID      INT);"""
)

conn.execute(
    "INSERT INTO DATAINFO (ID,NAME,SOURCE,FILELIST,SPEAKERID) \
      VALUES (1, 'eminem', 'uberduck', '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/eminem_all_processed.txt', 0)"
)

# id doesnt need to be included necessarily but makes sense
cursor = conn.execute("SELECT ID, NAME, SOURCE, FILELIST from DATAINFO")
for row in cursor:
    print("ID = ", row[0])
    print("NAME = ", row[1])
    print("ADDRESS = ", row[2])
    print("SALARY = ", row[3])

In [ ]:
# this is an example of a speakerlist that would be loaded from a json
# a json could also be generated by querying the database e.g.
# subset_speakers(database, seed, speakerlist = None, nspeakers = None)
speakerlist = np.asarray([["vctk", "p302"], ["uberduck", "eminem"]])
print(speakerlist)

[['vctk' 'p302']
 ['uberduck' 'eminem']]


In [ ]:
# now call get_filelist(database, speakerjson)
# this list (filelist locations) would be got from database
metalist_dir = "/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/metadata_collections"
metalist_files = os.listdir(metalist_dir)
# print(metalist_files)
train_ratios = np.ones(4) * 1.0
print(np.load(metalist_dir + "/" + metalist_files[0]))
print(np.load(metalist_dir + "/libritts_processed_file.npy"))
print(np.load(metalist_dir + "/uberduck_processed_files.npy", allow_pickle=True))
print(
    np.load(
        "/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/metadata_collections/vctk_processed_file.npy"
    )
)

In [ ]:
# now synthesize selected speakers from filelists in database and synthesize selected (e.g. [2,4,5])
# something like the following, but with the ability to take only rows of multispeaker datasets corresponding to particular speakers
filelists = np.asarray([])
# files = np.asarray([])
for r in range(4):
    files = np.load(metalist_dir + "/" + metalist_files[r], allow_pickle=True)
    filelist = np.asarray([])

    if files.ndim > 0:
        nfiles = files.shape[0]
        for s in range(nfiles):
            filelist = np.append(filelist, files[s])
    else:
        filelist = files
    filelists = np.append(filelists, filelist)

print(filelists)
dd = synthesize_speakerids2(filelists, 1)
ad = list(dd.values())
ad2 = [ad[i] for i in [2, 4, 5]]
alldata = pd.concat(ad2)

['/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/lj7all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/librittsall_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/eminem_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/ben-shapiro_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/jay-z_all_processed.txt'
 '/mnt/disks/uberduck-experiments-v0/uberduck-ml-dev/experiments/processed_metadata/vctkall_processed.txt']


/opt/conda/lib/python3.7/site-packages/numpy/lib/arraysetops.py:565: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  mask &= (ar1 != a)


In [ ]:
print(alldata)

                                                       0  \
0      /mnt/disks/uberduck-experiments-v0/data/uberdu...   
1      /mnt/disks/uberduck-experiments-v0/data/uberdu...   
2      /mnt/disks/uberduck-experiments-v0/data/uberdu...   
3      /mnt/disks/uberduck-experiments-v0/data/uberdu...   
4      /mnt/disks/uberduck-experiments-v0/data/uberdu...   
...                                                  ...   
43880  /mnt/disks/uberduck-experiments-v0/data/vctk/w...   
43881  /mnt/disks/uberduck-experiments-v0/data/vctk/w...   
43882  /mnt/disks/uberduck-experiments-v0/data/vctk/w...   
43883  /mnt/disks/uberduck-experiments-v0/data/vctk/w...   
43884  /mnt/disks/uberduck-experiments-v0/data/vctk/w...   

                                                       1    2  
0            It's like this and like that and like this.    0  
1      I'm the illest rapper to hold the cordless, pa...    0  
2      I'm meaner in action than Roscoe beatin' James...    0  
3      I grew up in a w